In [1]:
import fitz
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from concurrent.futures import ThreadPoolExecutor

/home/user1/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
##%%writefile func.py

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text("text")
    return text

def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks


def vectorize_chunks(chunks, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name).to("cuda")
    embeddings = model.encode(chunks, convert_to_tensor=True)
    
    return embeddings.cpu().detach().to(torch.float32).numpy()


def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index


def search_index(query, index, embedding_model, chunks, top_k=5):

    query_vector = embedding_model.encode([query], convert_to_numpy=True)  
    
    distances, indices = index.search(query_vector, top_k)
    
    return [(chunks[i], distances[0][idx]) for idx, i in enumerate(indices[0])]

def generate_answer_with_vikhr(retrieved_chunks, query):
    model = AutoModelForCausalLM.from_pretrained(
        "Vikhrmodels/Vikhr-7B-instruct_0.4",
        device_map="auto",
        torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-7B-instruct_0.4")

    combined_chunks = " ".join([chunk[0] for chunk in retrieved_chunks[:3]])  # Используем только 3 наиболее релевантных чанка

    prompt = f"Context: {combined_chunks}\n\nQuestion: {query}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    output = model.generate(
        **inputs,
        max_length=4064,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.3,
    )

    generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_answer

def main(pdf_path, query):
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text, chunk_size=500)

    embeddings = vectorize_chunks(chunks)

    index = build_faiss_index(embeddings)

    model = SentenceTransformer('all-MiniLM-L6-v2')
    results = search_index(query, index, model, chunks)
    
    answer = generate_answer_with_vikhr(results, query)
    
    print(f"Answer: {answer}")

In [5]:
pdf_path = "dogovor.pdf"
query = "Что нужно сделать, чтобы получить абонемент в спортивный зал?"

main(pdf_path, query)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Token indices sequence length is longer than the specified maximum sequence length for this model (2371 > 1512). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 